In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [3]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_Home', '_Away'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_Home != joined.TEAM_ID_Away]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_Home.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result


In [22]:
%%writeandexecute -i combine_team_games combine.py
    def combine_team_games(df, keep_method='home'):
    
    # Join every row to all others with the same game ID.
        joined = pd.merge(df, df, suffixes=['_Home', '_Away'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
        result = joined[joined.TEAM_ID_Home != joined.TEAM_ID_Away]
    # Take action based on the keep_method flag.
        if keep_method is None:
        # Return all the rows.
            pass
        elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
            result = result[result.MATCHUP_Home.str.contains(' vs. ')]
        elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
            result = result[result.MATCHUP_A.str.contains(' @ ')]
        elif keep_method.lower() == 'winner':
            result = result[result.WL_A == 'W']
        elif keep_method.lower() == 'loser':
            result = result[result.WL_A == 'L']
        else:
            raise ValueError(f'Invalid keep_method: {keep_method}')
        return result


In [4]:
with open('data/pickles/boxscoreadv21.p', 'rb') as readfile: 
    bs21 = pickle.load(readfile)
with open('data/pickles/fourfactors21.p', 'rb') as readfile: 
    ff21 = pickle.load(readfile)
with open('data/pickles/season21.p', 'rb') as readfile: 
    season21 = pickle.load(readfile)

with open('data/pickles/boxscoreadv20.p', 'rb') as readfile: 
    bs20 = pickle.load(readfile)
with open('data/pickles/fourfactors20.p', 'rb') as readfile: 
    ff20 = pickle.load(readfile)
with open('data/pickles/season20.p', 'rb') as readfile: 
    season20 = pickle.load(readfile)

with open('data/pickles/boxscoreadv19.p', 'rb') as readfile: 
    bs19 = pickle.load(readfile)
with open('data/pickles/fourfactors19.p', 'rb') as readfile: 
    ff19 = pickle.load(readfile)
with open('data/pickles/season19.p', 'rb') as readfile: 
    season19 = pickle.load(readfile)

with open('data/pickles/boxscoreadv18.p', 'rb') as readfile: 
    bs18 = pickle.load(readfile)
with open('data/pickles/fourfactors18.p', 'rb') as readfile: 
    ff18 = pickle.load(readfile)
with open('data/pickles/season18.p', 'rb') as readfile: 
    season18 = pickle.load(readfile)

with open('data/pickles/boxscoreadv17.p', 'rb') as readfile: 
    bs17 = pickle.load(readfile)
with open('data/pickles/fourfactors17.p', 'rb') as readfile: 
    ff17 = pickle.load(readfile)
with open('data/pickles/season17.p', 'rb') as readfile: 
    season17 = pickle.load(readfile)

In [9]:
from nba_api.stats.endpoints import boxscorefourfactorsv2
fix = boxscorefourfactorsv2.BoxScoreFourFactorsV2(game_id = '0022001069')
fixdf = fix.get_data_frames()[1]


ff21.reset_index(drop=True, inplace=True)

ff21.loc[0:1]=fixdf

In [30]:
def combine_dfs(season_df, bs_df, ff_df):
    merge1 = pd.merge(season_df, bs_df, on=['GAME_ID', 'TEAM_ID']).drop(labels=['TEAM_NAME_y', 'TEAM_ABBREVIATION_y', 'MIN_y', 'TEAM_CITY'], axis=1)
    merge2 = pd.merge(merge1, ff_df, on = ['GAME_ID', 'TEAM_ID']).drop(labels=['TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'MIN','EFG_PCT_x', 'TM_TOV_PCT_x'], axis=1)
    df = combine_team_games(merge2, keep_method='home')
    df.set_index(pd.to_datetime(df['GAME_DATE']), drop=True, inplace=True)
    df.sort_index(inplace=True)
    return df
    

In [31]:
def combine_dfs_seperate(season_df, bs_df, ff_df):
    merge1 = pd.merge(season_df, bs_df, on=['GAME_ID', 'TEAM_ID']).drop(labels=['TEAM_NAME_y', 'TEAM_ABBREVIATION_y', 'MIN_y', 'TEAM_CITY'], axis=1)
    merge2 = pd.merge(merge1, ff_df, on = ['GAME_ID', 'TEAM_ID']).drop(labels=['TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'MIN','EFG_PCT_x', 'TM_TOV_PCT_x'], axis=1)
    merge2.sort_values(by=["GAME_DATE", 'GAME_ID'], ascending=[1,1], inplace=True)
    merge2.reset_index(inplace=True, drop=True)
    return merge2

In [32]:
splitdf21 = combine_dfs_seperate(season21, bs21, ff21)
splitdf20 = combine_dfs_seperate(season20, bs20, ff20)
splitdf19 = combine_dfs_seperate(season19, bs19, ff19)
splitdf18 = combine_dfs_seperate(season18, bs18, ff18)
splitdf17 = combine_dfs_seperate(season17, bs17, ff17)

In [33]:
with open('data/pickles/splitdf21.p', 'wb') as writefile: 
    pickle.dump(splitdf21, writefile)
    
with open('data/pickles/splitdf20.p', 'wb') as writefile: 
    pickle.dump(splitdf20, writefile)
    
with open('data/pickles/splitdf19.p', 'wb') as writefile: 
    pickle.dump(splitdf19, writefile)
    
with open('data/pickles/splitdf18.p', 'wb') as writefile: 
    pickle.dump(splitdf18, writefile)
    
with open('data/pickles/splitdf17.p', 'wb') as writefile: 
    pickle.dump(splitdf17, writefile)

In [34]:
splitdf19.head(14)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION_x,TEAM_NAME_x,GAME_ID,GAME_DATE,MATCHUP,WL,MIN_x,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT_x,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,EFG_PCT_y,FTA_RATE,TM_TOV_PCT_y,OREB_PCT_y,OPP_EFG_PCT,OPP_FTA_RATE,OPP_TOV_PCT,OPP_OREB_PCT
0,22018,1610612755,PHI,Philadelphia 76ers,0021800001,2018-10-16,PHI @ BOS,L,239,87,34,87,0.391,5,26,0.192,14,23,0.609,6,41,47,18,8,5,16,20,-18.0,81.2,82.1,98.9,100.0,-17.7,-17.9,0.529,1.13,13.7,0.175,0.750,0.460,14.937,0.448,1.0,0.200,106.64,105.5,87.92,106,0.405,0.420,0.264,0.149,0.105,0.490,0.144,0.141,0.250
1,22018,1610612738,BOS,Boston Celtics,0021800001,2018-10-16,BOS vs. PHI,W,240,105,42,97,0.433,11,37,0.297,10,14,0.714,12,43,55,21,7,5,14,20,18.0,98.9,100.0,81.2,82.1,17.7,17.9,0.500,1.40,15.1,0.250,0.825,0.540,14.130,0.509,1.0,0.198,106.64,105.5,87.92,105,0.595,0.490,0.144,0.141,0.214,0.420,0.264,0.149,0.175
2,22018,1610612760,OKC,Oklahoma City Thunder,0021800002,2018-10-16,OKC @ GSW,L,240,100,33,91,0.363,10,37,0.270,24,37,0.649,16,29,45,21,12,6,14,21,-8.0,94.1,97.1,101.0,104.9,-6.9,-7.8,0.636,1.40,14.7,0.348,0.585,0.454,14.114,0.466,1.0,0.200,106.60,103.0,85.83,103,0.418,0.418,0.407,0.141,0.242,0.479,0.189,0.196,0.415
3,22018,1610612744,GSW,Golden State Warriors,0021800002,2018-10-16,GSW vs. OKC,W,241,108,42,95,0.442,7,26,0.269,17,18,0.944,17,41,58,28,7,7,21,29,8.0,101.0,104.9,94.1,97.1,6.9,7.8,0.667,1.33,18.4,0.415,0.652,0.546,19.641,0.525,1.0,0.199,106.60,103.0,85.83,103,0.582,0.479,0.189,0.196,0.321,0.418,0.407,0.141,0.348
4,22018,1610612749,MIL,Milwaukee Bucks,0021800003,2018-10-17,MIL @ CHA,W,240,113,42,85,0.494,14,34,0.412,15,20,0.750,11,46,57,26,5,4,21,25,1.0,108.9,109.7,108.0,107.7,0.8,2.0,0.619,1.24,18.5,0.304,0.807,0.583,20.231,0.602,1.0,0.200,103.74,103.5,86.25,103,0.527,0.576,0.235,0.202,0.239,0.533,0.239,0.106,0.193
5,22018,1610612766,CHA,Charlotte Hornets,0021800003,2018-10-17,CHA vs. MIL,L,241,112,41,92,0.446,16,38,0.421,14,22,0.636,9,32,41,21,8,9,11,19,-1.0,108.0,107.7,108.9,109.7,-0.8,-2.0,0.512,1.91,15.7,0.193,0.696,0.417,10.610,0.551,1.0,0.199,103.74,103.5,86.25,104,0.473,0.533,0.239,0.106,0.158,0.576,0.235,0.202,0.304
6,22018,1610612765,DET,Detroit Pistons,0021800004,2018-10-17,DET vs. BKN,W,240,103,39,92,0.424,6,24,0.250,19,22,0.864,14,32,46,21,5,5,14,20,3.0,98.4,103.0,94.6,99.0,3.8,4.0,0.538,1.24,15.0,0.345,0.739,0.525,16.240,0.506,1.0,0.192,105.18,100.5,83.75,100,0.477,0.457,0.239,0.162,0.255,0.518,0.268,0.180,0.261
7,22018,1610612751,BKN,Brooklyn Nets,0021800004,2018-10-17,BKN @ DET,L,240,100,40,82,0.488,5,27,0.185,15,22,0.682,5,34,39,28,9,5,17,23,-3.0,94.6,99.0,98.4,103.0,-3.8,-4.0,0.700,1.47,20.2,0.261,0.655,0.475,17.979,0.545,1.0,0.195,105.18,100.5,83.75,101,0.523,0.518,0.268,0.180,0.109,0.457,0.239,0.162,0.345
8,22018,1610612763,MEM,Memphis Grizzlies,0021800005,2018-10-17,MEM @ IND,L,240,83,25,84,0.298,10,29,0.345,23,28,0.821,7,21,28,16,11,3,7,18,-28.0,83.6,87.4,116.0,115.6,-32.4,-28.3,0.640,1.60,13.1,0.242,0.564,0.366,10.068,0.431,1.0,0.196,97.52,95.5,79.58,95,0.286,0.357,0.333,0.101,0.113,0.627,0.157,0.209,0.436
9,22018,1610612754,IND,Indiana Pacers,0021800005,2018-10-17,IND vs. MEM,W,240,111,47,83,0.566,10,26,0.385,7,13,0.538,13,44,57,29,2,7,20,24,28.0,116.0,115.6,83.6,87.4,32.4,28.3,0.617,1.45,21.1,0.436,0.758,0.634,20.894,0.626,1.0,0.199,97.52,95.5,79.58,96,0.714,0.627,0.157,0.209,0.333,0.357,0.333,0.101,0.242


In [35]:
df21 = combine_dfs(season21, bs21, ff21)
df20 = combine_dfs(season20, bs20, ff20)
df19 = combine_dfs(season19, bs19, ff19)
df18 = combine_dfs(season18, bs18, ff18)
df17 = combine_dfs(season17, bs17, ff17)

In [36]:
with open('data/pickles/df21.p', 'wb') as writefile: 
    pickle.dump(df21, writefile)
    
with open('data/pickles/df20.p', 'wb') as writefile: 
    pickle.dump(df20, writefile)
    
with open('data/pickles/df19.p', 'wb') as writefile: 
    pickle.dump(df19, writefile)
    
with open('data/pickles/df18.p', 'wb') as writefile: 
    pickle.dump(df18, writefile)
    
with open('data/pickles/df17.p', 'wb') as writefile: 
    pickle.dump(df17, writefile)
